In [ ]:
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  20.8M      0  0:00:03  0:00:03 --:--:-- 20.8M


In [ ]:
# !rm -r aclImdb/train/unsup

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
!rm -r aclImdb/val/neg
for category in ('neg', 'pos') :
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]

  for fname in val_files :
    shutil.move(train_dir / category / fname,
                val_dir / category / fname)

rm: cannot remove 'aclImdb/val/neg': No such file or directory


In [ ]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size = batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size = batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size = batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds :
  print('inputs.shape : ', inputs.shape)
  print('inputs.dtype : ', inputs.dtype)
  print('targets.shape : ', targets.shape)
  print('targets.dtype : ', targets.dtype)
  print('inputs[0] : ', inputs[0])
  print('targets[0] : ', targets[0])
  break

inputs.shape :  (32,)
inputs.dtype :  <dtype: 'string'>
targets.shape :  (32,)
targets.dtype :  <dtype: 'int32'>
inputs[0] :  tf.Tensor(b"This was a strong Poirot/Suchet, television mystery selection. The characters were vivid and well-acted. The plot and the main setting--a student hostel-- were excellent. Japp was nothing special but for me did not distract from story. One significant point, many Poirot watchers don't recognize good acting or good characterization. I also think they are rather harsh in their judgments of some of the Poirot mysteries. Finally, I have read few Christie novels--none in recent years-- and find it annoying that so many viewers are upset about changes from the novel. Please, viewers, consider what is presented to you on film, not what you think should be there. That said, the Poirot mysteries vary in quality, but not as much as reviewers and raters would have you believe. With the singular exception of The Five Little Pigs which was fabulous in plot, chara

In [ ]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens = 20000,
    output_mode = 'multi_hot',
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

이진 유니그램 데이터셋의 출력 확인하기

In [ ]:
for inputs, targets in binary_1gram_train_ds :
  print('inputs.shape : ', inputs.shape)
  print('inputs.dtype : ', inputs.dtype)
  print('targets.shape : ', targets.shape)
  print('targets.dtype : ', targets.dtype)
  print('inputs[0] : ', inputs[0])
  print('targets[0] : ', targets[0])
  break

inputs.shape :  (32, 20000)
inputs.dtype :  <dtype: 'float32'>
targets.shape :  (32,)
targets.dtype :  <dtype: 'int32'>
inputs[0] :  tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0] :  tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
from tensorflow.keras import layers

def get_model(max_tokens = 20000, hidden_dim = 16) :
  inputs = keras.Input(shape = (max_tokens,))
  x = layers.Dense(hidden_dim, activation = 'relu')(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation = 'sigmoid')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer = 'rmsprop',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])
  return model

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_1gram.keras',
                                    save_best_only = True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data = binary_1gram_val_ds.cache(), # 데이터를 메모리에 캐시하여 데이터 로딩 속도를 높여줌
          epochs = 10,
          callbacks = callbacks)
model = keras.models.load_model('binary_1gram.keras')
print(f'테스트 정확도 : {model.evaluate(binary_1gram_test_ds)[1] : .3f}')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 9s 5ms/step - loss: 0.4009 - accuracy: 0.8330 - val_loss: 0.3046 - val_accuracy: 0.8778
Epoch 2/10
625/625 [=======================

In [ ]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_length
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)
int_val_ds = val_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)
int_test_ds = test_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls = 4
)

In [ ]:
import tensorflow as tf

inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = tf.one_hot(inputs, depth = max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7da2f7030850>>

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras',
                                    save_best_only = True)
]
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10,
          callbacks = callbacks)
model = keras.models.load_model('one_hot_bidir_lstm.keras')
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1] : .3f}')

Epoch 1/10
625/625 [==============================] - 159s 254ms/step - loss: 0.5154 - accuracy: 0.7630 - val_loss: 0.3799 - val_accuracy: 0.8544
Epoch 2/10
625/625 [==============================] - 158s 253ms/step - loss: 0.3409 - accuracy: 0.8766 - val_loss: 0.4359 - val_accuracy: 0.8370
Epoch 3/10
625/625 [==============================] - 158s 252ms/step - loss: 0.2862 - accuracy: 0.9016 - val_loss: 0.3339 - val_accuracy: 0.8688
Epoch 4/10
625/625 [==============================] - 158s 252ms/step - loss: 0.2398 - accuracy: 0.9186 - val_loss: 0.3452 - val_accuracy: 0.8734
Epoch 5/10
625/625 [==============================] - 158s 252ms/step - loss: 0.2108 - accuracy: 0.9287 - val_loss: 0.3067 - val_accuracy: 0.8840
Epoch 6/10
625/625 [==============================] - 158s 253ms/step - loss: 0.1897 - accuracy: 0.9389 - val_loss: 0.3276 - val_accuracy: 0.8732
Epoch 7/10
625/625 [==============================] - 158s 252ms/step - loss: 0.1706 - accuracy: 0.9464 - val_loss: 0.3660 -

In [ ]:
embedding_layer = layers.Embedding(input_dim = max_tokens, output_dim = 256)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary

callbacks = [
    keras.callbacks.ModelCheckpoint('embedding_bidir_lstm.keras',
                                    save_best_only = True)
]
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10,
          callbacks = callbacks)
model = keras.models.load_model('embedding_bidir_lstm.keras')
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1] : .3f}')

Epoch 1/10
625/625 [==============================] - 91s 140ms/step - loss: 0.5177 - accuracy: 0.7434 - val_loss: 0.3838 - val_accuracy: 0.8480
Epoch 2/10
625/625 [==============================] - 45s 72ms/step - loss: 0.3453 - accuracy: 0.8698 - val_loss: 0.3814 - val_accuracy: 0.8384
Epoch 3/10
625/625 [==============================] - 37s 60ms/step - loss: 0.2748 - accuracy: 0.9024 - val_loss: 0.3857 - val_accuracy: 0.8360
Epoch 4/10
625/625 [==============================] - 32s 52ms/step - loss: 0.2299 - accuracy: 0.9201 - val_loss: 0.3537 - val_accuracy: 0.8716
Epoch 5/10
625/625 [==============================] - 28s 45ms/step - loss: 0.1926 - accuracy: 0.9374 - val_loss: 0.3778 - val_accuracy: 0.8748
Epoch 6/10
625/625 [==============================] - 26s 42ms/step - loss: 0.1720 - accuracy: 0.9427 - val_loss: 0.3722 - val_accuracy: 0.8632
Epoch 7/10
625/625 [==============================] - 26s 41ms/step - loss: 0.1458 - accuracy: 0.9527 - val_loss: 0.4163 - val_accuracy

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

--2023-07-14 03:16:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-14 03:16:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-14 03:16:06--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# import numpy as np

# path_to_glove_file = 'glove.6B.100d.txt'
# embeddings_index = {}

# with open(path_to_glove_file) as f :
#   for line in f :
#     word, coefs = line.split(maxsplit = 1)
#     coefs = np.fromstring(coefs, 'f', sep = '')
#     embeddings_index[word] = coefs

# print(f'단어 벡터 개수 : {len(embeddings_index)}')

In [ ]:
path_to_glove_file = 'glove.6B.100d.txt'
embeddings_index = {}

with open(path_to_glove_file, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.fromstring(' '.join(values[1:]), dtype=np.float32, sep=' ')
        embeddings_index[word] = coefs

print('단어 벡터 개수:', len(embeddings_index))

단어 벡터 개수: 400000


In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
  if i < max_tokens :
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None :
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer = keras.initializers.Constant(embedding_matrix),
    trainable = False,
    mask_zero = True,
)

In [ ]:
inputs = keras.Input(shape = (None,), dtype = 'int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary

callbacks = [
    keras.callbacks.ModelCheckpoint('glove_embeddings_sequence_model',
                                    save_best_only = True)
]
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10,
          callbacks = callbacks)
model = keras.models.load_model('glove_embeddings_sequence_model')
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1] : .3f}')

Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.5808 - accuracy: 0.6924

625/625 [==============================] - 71s 99ms/step - loss: 0.5807 - accuracy: 0.6926 - val_loss: 0.5884 - val_accuracy: 0.6606
Epoch 2/10
624/625 [============================>.] - ETA: 0s - loss: 0.4611 - accuracy: 0.7882

625/625 [==============================] - 59s 95ms/step - loss: 0.4609 - accuracy: 0.7882 - val_loss: 0.4131 - val_accuracy: 0.8122
Epoch 3/10
625/625 [==============================] - ETA: 0s - loss: 0.4103 - accuracy: 0.8182

625/625 [==============================] - 59s 95ms/step - loss: 0.4103 - accuracy: 0.8182 - val_loss: 0.4017 - val_accuracy: 0.8210
Epoch 4/10
625/625 [==============================] - ETA: 0s - loss: 0.3776 - accuracy: 0.8353

625/625 [==============================] - 59s 95ms/step - loss: 0.3776 - accuracy: 0.8353 - val_loss: 0.3608 - val_accuracy: 0.8414
Epoch 5/10
624/625 [============================>.] - ETA: 0s - loss: 0.3499 - accuracy: 0.8511

625/625 [==============================] - 61s 97ms/step - loss: 0.3502 - accuracy: 0.8510 - val_loss: 0.3601 - val_accuracy: 0.8434
Epoch 6/10
625/625 [==============================] - ETA: 0s - loss: 0.3276 - accuracy: 0.8648

625/625 [==============================] - 60s 96ms/step - loss: 0.3276 - accuracy: 0.8648 - val_loss: 0.3375 - val_accuracy: 0.8538
Epoch 7/10
624/625 [============================>.] - ETA: 0s - loss: 0.3061 - accuracy: 0.8724

625/625 [==============================] - 60s 96ms/step - loss: 0.3060 - accuracy: 0.8724 - val_loss: 0.3264 - val_accuracy: 0.8658
Epoch 8/10
625/625 [==============================] - 26s 42ms/step - loss: 0.2911 - accuracy: 0.8804 - val_loss: 0.3268 - val_accuracy: 0.8624
Epoch 9/10
624/625 [============================>.] - ETA: 0s - loss: 0.2800 - accuracy: 0.8859

625/625 [==============================] - 60s 96ms/step - loss: 0.2802 - accuracy: 0.8858 - val_loss: 0.3188 - val_accuracy: 0.8718
Epoch 10/10
782/782 [==============================] - 17s 18ms/step - loss: 0.3029 - accuracy: 0.8693
테스트 정확도 :  0.869
